In [1]:
import string
import pandas as pd
from lxml import html
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from IPython.core.display import display, HTML

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

pd.set_option('display.max_colwidth', -1)

def decodeEmail(e):
    de = ""
    k = int(e[:2], 16)

    for i in range(2, len(e)-1, 2):
        de += chr(int(e[i:i+2], 16)^k)

    return de


def parse_infopage(url_link, data):
    
    req = Request(url_link, headers={'User-Agent': 'Mozilla/5.0'})
    webpage_authorised = urlopen(req).read()
    soup_authorised = BeautifulSoup(webpage_authorised.decode("utf-8"), "html.parser")
    
    
    cols=["CID"]
    
    cols.append("Company")
    for name in soup_authorised.findAll("h1",class_="RecordName"):
        data.append(name.text)
        
    for div in soup_authorised.findAll("div", class_="address section"):
        for h3 in div.findAll("h3",class_="addressheader"):
            if h3.text == "Principal place of business":
                for sp in div.findAll("span", class_="addresslabel"):
                    cols.append(sp.text.strip())
                    
                for d in div.findAll("div", class_="addressvalue"):
                    data.append(' '.join(d.text.split()))
        
                for sp in div.findAll("span", class_="addressvalue"):
                    if "email" in sp.text.strip():
                        email = sp.find("a", class_="__cf_email__")
                        data.append(decodeEmail(email['data-cfemail']))
                    else:
                        data.append(sp.text.strip())
    
    cols.append("Status")
    for stat in soup_authorised.findAll("span",class_="statusbox"):
            if "No longer authorised" in stat.text:
                data.append("Not Authorised")
            elif "Authorised" in stat.text:
                data.append("Authorised")
            
    return data, cols

ModuleNotFoundError: No module named 'selenium'

In [2]:
fp = open("cid.txt","r")
cids = fp.readlines()
cids = [ix.strip("\n") for ix in cids]

info = []

for cid in cids:
    
    driver = webdriver.Chrome("/home/wns/chromedriver")
    driver.get("https://register.fca.org.uk/")

    url=""
    data=[]
    
    driver.find_element_by_id('j_id0:j_id1:j_id33:j_id34:registersearch:j_id36:searchBox').send_keys(cid);
    button = driver.find_element_by_id('j_id0:j_id1:j_id33:j_id34:registersearch:j_id36:j_id39')
    button.click()

    try:
        if WebDriverWait(driver, 10).until(EC.url_contains("https://register.fca.org.uk/ShPo")):
            url = driver.current_url
            #print(url)
            driver.quit()

    except TimeoutException:
        if WebDriverWait(driver, 10).until(EC.url_contains("https://register.fca.org.uk/shpo_")):
            url = driver.current_url
            #print(url)
            driver.quit()



    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    if "https://register.fca.org.uk/shpo_" in url:

        soup = BeautifulSoup(webpage.decode("utf-8"), "html.parser")
        flag = 0
        base_weblink = "https://register.fca.org.uk/"
        authorised_link = ""

        for table in soup.findAll("table", id="SearchResults"):
            for row in table.findAll("tr"):
                for col in row.findAll("td"):
                    for sp in col.findAll("span", class_="CurrentStatus Authorised Authorised search_popover"):
                        if col.text == "Authorised":
                            flag = 1
                if flag == 1:
                    for name in row.findAll("td", class_="ResultName"):
                        for a in name.findAll("a"):
                            authorised_link = base_weblink + a["href"]
                        flag=0
        
        data.append(cid)                
        data,cols = parse_infopage(authorised_link, data)
        info.append(data)
        
    elif "https://register.fca.org.uk/ShPo" in url:
        
        data.append(cid)
        data,cols = parse_infopage(url, data)
        info.append(data)
        
df = pd.DataFrame(info,columns = cols)

writer = pd.ExcelWriter('companies_info.xlsx') 
df.to_excel(writer,sheet_name='CID_Info')
writer.save()
df

,CID,Company,Address:,Phone:,Fax:,Email:,Website:,Status
0,310164,A-Plan Holdings,2 Des Roches square Witney Oxfordshire OX28 4LE UNITED KINGDOM,44 1993893311,44 1993893310,cas@a-plan.co.uk,www.a-plan.co.uk,Authorised
1,307494,A.I.F. Ltd,14 Falmouth Avenue Highams Park London E4 9QR UNITED KINGDOM,44 02089237558,44 02082817770,r.holmes@alfa-insurance.co.uk,,Authorised
2,305637,A H Bell & Co (Insurance Brokers) Ltd,The Business & Media Centre The County Ground Nottingham Road Derby Derbyshire DE21 6DA UNITED KINGDOM,44 1332372111,+44,saltr@bellbrokers.co.uk,www.bellbrokers.co.uk,Not Authorised
3,519675,A J Insurance Service Limited,Fleet House 148 Dock Road Tilbury Essex RM18 7BS UNITED KINGDOM,+44 08448806048,+44 08448806815,paula@ajinsurance.co.uk,www.ajinsurance.co.uk,Authorised
